In [1]:
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append('/content/drive/My Drive/dp_tomastik/code')
    !bash "/content/drive/My Drive/dp_tomastik/code/scripts/install_libs.sh"
    
import SimpleITK as sitk
import matplotlib.pyplot as plt
import torch
import os
import numpy as np
import pandas as pd
import logging
import datetime

from torchio import RandomAffine, Compose, ZNormalization
from operator import itemgetter
from IPython.display import display, Markdown
from ipywidgets import widgets

from src.helpers import preview_3d_image
from src.helpers import show_cuda_usage, preview_model_dataset_pred, preview_dataset
from src.helpers import get_threshold_info_df, get_rescaled_preds
from src.helpers import compare_prediction_with_ground_true, compare_one_prediction_with_ground_true
from src.helpers import get_img_outliers_pixels, get_raw_with_prediction
from src.helpers import get_rescaled_pred
from src.helpers import get_transformed_label_np, create_regis_trans_list, trans_list

from src.dataset import HaNOarsDataset, transform_input_with_registration, get_norm_transform
from src.dataset import get_full_res_cut, get_cut_lists, OARS_LABELS, get_dataset, get_dataset_info, get_dataset_transform
from src.dataset import split_dataset, copy_split_dataset

from src.model_and_training import prepare_model, train_loop, show_model_info, load_checkpoint_model_info
from src.model_and_training.getters.get_device import get_device
from src.model_and_training.architectures.unet_architecture_v3v1 import UNetV3v1

from src.consts import DATASET_MAX_BOUNDING_BOX, DESIRE_BOUNDING_BOX_SIZE
  
torch.manual_seed(20)
logging.basicConfig(filename='logs/model3v1_all_organs_jupyter.log', level=logging.DEBUG)

print('Dataset biggest bounding box wihtout spinal cord', DATASET_MAX_BOUNDING_BOX)
print('Cut target size', DESIRE_BOUNDING_BOX_SIZE)
print('Done Init')

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits

Dataset biggest bounding box wihtout spinal cord [56, 177, 156]
Cut target size [72, 192, 168]
Done Init


In [2]:
def get_possible_models(oar_key):
    possible_models = [folder_name for folder_name in os.listdir('./models') if oar_key in folder_name]    
    
    return possible_models

# Training all organs models

In [3]:
data_path = f'./data/HaN_OAR_cut_72_192_168'
cut_dataset = HaNOarsDataset(data_path, size=50, load_images=False)
cut_dataset.load_from_file(data_path)
cut_dataset_obj = split_dataset(cut_dataset, train_size=40, valid_size=5, test_size=5)
cut_train_dataset, cut_valid_dataset, cut_test_dataset = itemgetter(*['train_dataset', 'valid_dataset', 'test_dataset'])(cut_dataset_obj)

In [4]:
get_dataset_info(cut_dataset, cut_dataset_obj)
preview_dataset(cut_dataset)

train 40, valid_size 5, test 5, full 50
train indices [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18, 20, 21, 22, 23, 24, 28, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
valid indices [6, 13, 19, 25, 38]
test indices [16, 26, 27, 29, 39]
data max 3071, min -1024
label max 22, min 0


Output()

checking list used for training single models

training each model

In [5]:
filter_labels = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels['SPINAL_CORD']

tmp_list = list(filter_labels.items())
labels_list = [tmp_list[10]]
for OAR_KEY, OAR_VALUE in labels_list:
    print(f"{OAR_KEY}, {OAR_VALUE}")

PITUITARY, 11


In [6]:
TRAIN_MODELS = False
if TRAIN_MODELS:
    for OAR_KEY, OAR_VALUE in labels_list:
        cut_dataset.set_output_label(OAR_VALUE)
        log_date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        model_name = f'{log_date}_3d_unet_lowres_model3v1__cloud-{OAR_KEY}'

        print(f'Training model with dataset label \'{OAR_KEY}\', value \'{OAR_VALUE}\'')
        print(f'folder \'{model_name}\'')
        cut_model_info = prepare_model(epochs=175,
                                       learning_rate=3e-4,
                                       in_channels=8,
                                       input_data_channels=1,
                                       output_label_channels=1,
                                       dropout_rate=0.2,
                                       train_batch_size=2,
                                       model_name=model_name,
                                       train_dataset=cut_train_dataset, 
                                       valid_dataset=cut_valid_dataset, 
                                       test_dataset=cut_test_dataset,
                                       model_class=UNetV3v1)
        show_model_info(cut_model_info)
        print('\n\n')
        train_loop(cut_model_info)
        print('\n\n')

        # clearing memory
        del cut_model_info
        torch.cuda.empty_cache()

## DSC

In [7]:
filter_labels = OARS_LABELS.OARS_LABELS_DICT
if 'SPINAL_CORD' in filter_labels:
    del filter_labels['SPINAL_CORD']

tmp_list = list(filter_labels.items())
labels_list = [tmp_list[10]]
for OAR_KEY, OAR_VALUE in labels_list:
    print(f"{OAR_KEY}, {OAR_VALUE}")

PITUITARY, 11


### loading models to CPU 

In [8]:
models = dict()
for OAR_KEY, OAR_VALUE in labels_list:
    possible_models = get_possible_models(f"model3v1__cloud-{OAR_KEY}")
    if len(possible_models) <= 0:
        print(f'{OAR_KEY} Model: No avaiable model')
        continue

    model_name = possible_models[0]
    print(f'{OAR_KEY} Model: Loading model {model_name}')

    # loading model checkpoint
    epoch = 175
    cut_model_info = load_checkpoint_model_info(model_name, epoch, cut_train_dataset, cut_valid_dataset, cut_test_dataset, model_class=UNetV3v1)

    # moving model to cpu/cuda with eval mode
    cut_model_info['device'] = 'cpu'
    cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
    cut_model_info['model'].eval()
    cut_model_info['model'].disable_tensorboard_writing = True
    
    models[OAR_KEY] = cut_model_info

PITUITARY Model: Loading model 20210425-143946_3d_unet_lowres_model3v1__cloud-PITUITARY


In [9]:
SHOW_DSC_INFO = True
if SHOW_DSC_INFO:
    info_per_organs_df = {}
    models_info = list()
    for OAR_KEY, OAR_VALUE in labels_list:
        if OAR_KEY not in models:
            print(f'{OAR_KEY} Model: No avaiable model')
            continue

        # getting model to gpu
        cut_model_info = models[OAR_KEY]
        cut_model_info['device'] = get_device()
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])
        cut_model_info['model'].eval()
        cut_model_info['model'].disable_tensorboard_writing = True

        # preparing dataset for comparison
        cut_dataset.set_output_label(OAR_VALUE)

        # calculating dsc predictions        
        info_df, preds, rescaled_preds = get_threshold_info_df(model=cut_model_info['model'], 
                                    dataset=cut_dataset, 
                                    device=cut_model_info['device'], 
                                    train_indices=cut_train_dataset.indices, 
                                    valid_indices=cut_valid_dataset.indices, 
                                    test_indices=cut_test_dataset.indices,
                                    step=0.5)
        info_per_organs_df[OAR_KEY] = info_df

        # moving model back to cpu
        cut_model_info['device'] = 'cpu'
        cut_model_info['model'] = cut_model_info['model'].to(cut_model_info['device'])

        # parsing data
        best_threshold_col = 'thres_rescaled_dsc_0.50'
        train_tmp_df = info_df[info_df['is_train']][best_threshold_col]
        valid_tmp_df = info_df[info_df['is_valid']][best_threshold_col]
        test_tmp_df = info_df[info_df['is_test']][best_threshold_col]
        
        train_dsc = train_tmp_df.mean()
        valid_dsc = valid_tmp_df.mean()
        test_dsc = test_tmp_df.mean()
        print(f'{OAR_KEY} Model: DSC train {round(train_dsc, 4)} valid {round(valid_dsc, 4)}')

        models_info.append({
            'oar_key': OAR_KEY,
            'model_name': model_name,
            # Train
            'train_dsc_mean': train_dsc,
            'train_dsc_std': train_tmp_df.std(),
            'train_dsc_median': train_tmp_df.median(),
            'train_dsc_min': train_tmp_df.min(),
            'train_dsc_max': train_tmp_df.max(),
            # Valid
            'valid_dsc_mean': valid_dsc,
            'valid_dsc_std': valid_tmp_df.std(),
            'valid_dsc_median': valid_tmp_df.median(),
            'valid_dsc_min': valid_tmp_df.min(),
            'valid_dsc_max': valid_tmp_df.max(),
            # Test
            'test_dsc_mean': test_dsc,
            'test_dsc_std': test_tmp_df.std(),
            'test_dsc_median': test_tmp_df.median(),
            'test_dsc_min': test_tmp_df.min(),
            'test_dsc_max': test_tmp_df.max(),
            # Both
            'train_valid_mean_delta': train_dsc - valid_dsc
        })

    models_info_df = pd.DataFrame(models_info)
    
    tmp_df = models_info_df[['model_name',
                             'train_dsc_mean', 'train_dsc_std', 
                             'valid_dsc_mean', 'valid_dsc_std', 
                             'test_dsc_mean', 'test_dsc_std']].copy()
    tmp_df['train_dsc_mean'] = (tmp_df['train_dsc_mean'] * 100).round(2)
    tmp_df['valid_dsc_mean'] = (tmp_df['valid_dsc_mean'] * 100).round(2)
    tmp_df['test_dsc_mean'] = (tmp_df['test_dsc_mean'] * 100).round(2)

    tmp_df['train_dsc_std'] = (tmp_df['train_dsc_std'] * 100).round(2)
    tmp_df['valid_dsc_std'] = (tmp_df['valid_dsc_std'] * 100).round(2)
    tmp_df['test_dsc_std'] = (tmp_df['test_dsc_std'] * 100).round(2)
    
    display(tmp_df.mean().round(2))
    display(tmp_df.round(2))
    display(tmp_df.sort_values(by=['train_dsc_std']).round(2))
    display(models_info_df.sort_values(by=['train_dsc_mean']).drop(columns=['model_name']).round(2))
    display(models_info_df.sort_values(by=['train_valid_mean_delta']).drop(columns=['model_name']).round(2))

PITUITARY Model: DSC train 0.655 valid 0.488


train_dsc_mean    65.50
train_dsc_std     19.29
valid_dsc_mean    48.80
valid_dsc_std     20.59
test_dsc_mean     31.63
test_dsc_std      28.35
dtype: float64

,model_name,train_dsc_mean,train_dsc_std,valid_dsc_mean,valid_dsc_std,test_dsc_mean,test_dsc_std
0,20210425-143946_3d_unet_lowres_model3v1__cloud...,65.5,19.29,48.8,20.59,31.63,28.35


,model_name,train_dsc_mean,train_dsc_std,valid_dsc_mean,valid_dsc_std,test_dsc_mean,test_dsc_std
0,20210425-143946_3d_unet_lowres_model3v1__cloud...,65.5,19.29,48.8,20.59,31.63,28.35


,oar_key,train_dsc_mean,train_dsc_std,train_dsc_median,train_dsc_min,train_dsc_max,valid_dsc_mean,valid_dsc_std,valid_dsc_median,valid_dsc_min,valid_dsc_max,test_dsc_mean,test_dsc_std,test_dsc_median,test_dsc_min,test_dsc_max,train_valid_mean_delta
0,PITUITARY,0.66,0.19,0.69,0.0,0.93,0.49,0.21,0.55,0.17,0.73,0.32,0.28,0.38,0.0,0.6,0.17


,oar_key,train_dsc_mean,train_dsc_std,train_dsc_median,train_dsc_min,train_dsc_max,valid_dsc_mean,valid_dsc_std,valid_dsc_median,valid_dsc_min,valid_dsc_max,test_dsc_mean,test_dsc_std,test_dsc_median,test_dsc_min,test_dsc_max,train_valid_mean_delta
0,PITUITARY,0.66,0.19,0.69,0.0,0.93,0.49,0.21,0.55,0.17,0.73,0.32,0.28,0.38,0.0,0.6,0.17


In [10]:
if SHOW_DSC_INFO:
    tmp_column = 'is_train' 
    
#     print('OARS_LABELS.PAROTID_GLAND_L')
#     tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.PAROTID_GLAND_L]]
#     display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    
#     print('OARS_LABELS.OPT_NERVE_L')
#     tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.OPT_NERVE_L]]
#     display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    
    print('OARS_LABELS.PITUITARY')
    tmp_df = info_per_organs_df[OARS_LABELS.OARS_LABELS_R_DICT[OARS_LABELS.PITUITARY]]
    # display(tmp_df[tmp_df[tmp_column]].sort_values(by='thres_rescaled_dsc_0.50'))
    display(tmp_df.sort_values(by='thres_rescaled_dsc_0.50'))

OARS_LABELS.PITUITARY


,dsc,rescaled_dsc,is_train,is_valid,is_test,thres_rescaled_dsc_0.00,thres_rescaled_dsc_0.50,thres_rescaled_dsc_1.00
index,,,,,,,,
27,0.001719,0.001719,False,False,True,0.000034,7.352941e-09,2.500000e-08
48,0.068419,0.073810,True,False,False,0.000011,5.263157e-08,7.692307e-08
29,0.026538,0.045741,False,False,True,0.000177,3.827752e-02,4.878049e-09
15,0.183015,0.188769,True,False,False,0.000026,1.500000e-01,3.333333e-08
19,0.160938,0.165876,False,True,False,0.000099,1.732284e-01,8.695652e-09
22,0.367698,0.367698,True,False,False,0.000024,3.684211e-01,3.571428e-08
39,0.383471,0.383471,False,False,True,0.000127,3.842365e-01,6.756757e-09
25,0.432045,0.432045,False,True,False,0.000139,4.264706e-01,6.211180e-09
42,0.447547,0.448720,True,False,False,0.000026,4.390244e-01,3.333333e-08


In [13]:
from matplotlib.patches import Patch


def preview_3d_image(img, figsize=(16, 16), show_index=None, show_slice=None, show_unique=False, minmax=False):
    if type(img) is sitk.SimpleITK.Image:
        img = sitk.GetArrayFromImage(img)

    max_slices = img.shape[0]

    def f(slice_index):
        fig = plt.figure(figsize=figsize, dpi=100)

        if minmax:
            plt.imshow(img[slice_index], vmin=np.min(img), vmax=np.max(img))
        else:
            plt.imshow(img[slice_index])

        print(f'Extension: index {show_index}, slice: {slice_index}')
        plt.title(f'Extension: index {show_index}, slice: {slice_index}')
        plt.legend(handles=[
            Patch(facecolor='red', label='Ground true'),
            Patch(facecolor='green', label='Intersection'),
            Patch(facecolor='blue', label='Model'),
        ])
        plt.axis('off')
        plt.show()

        if show_unique:
            print(f"debug: {img.min()}, {img.max()}")
            print(f"debug: unique {np.unique(img[slice_index])}")

    if show_slice:
        sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=show_slice)
    else:
        sliceSlider = widgets.IntSlider(min=0, max=max_slices - 1, step=1, value=(max_slices - 1) / 2)

    ui = widgets.VBox([widgets.HBox([sliceSlider])])
    out = widgets.interactive_output(f, {'slice_index': sliceSlider})
    # noinspection PyTypeChecker
    display(ui, out)

    
def show_comparision(dataset_image_input, dataset_image, prediction, figsize=(16,16), show_index=None, show_slice=None):
    intersection = dataset_image * prediction

    empty_compare_img = np.zeros((*dataset_image.shape, 3))

    tmp = dataset_image - intersection
    empty_compare_img[:, :, :, 0] = tmp
    empty_compare_img[:, :, :, 1] = intersection
    tmp = prediction - intersection
    empty_compare_img[:, :, :, 2] = tmp

    preview_3d_image(empty_compare_img, figsize=figsize, show_index=show_index, show_slice=show_slice)


    
    
figsize=(4,4)

show_index=6
zoom_weight = 74
zoom_weight_horiz = 62
dataset_image_input = (cut_dataset[show_index][0][0])[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]
dataset_image = (cut_dataset[show_index][1][0])[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]
prediction = (rescaled_preds[show_index][0] > 0.5)[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]

show_comparision(dataset_image_input, dataset_image, prediction, figsize=figsize, show_index=show_index, show_slice=48)
show_comparision(dataset_image_input, dataset_image, prediction, figsize=figsize, show_index=show_index, show_slice=49)


show_index=13
zoom_weight = 74
zoom_weight_horiz = 62
dataset_image_input = (cut_dataset[show_index][0][0])[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]
dataset_image = (cut_dataset[show_index][1][0])[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]
prediction = (rescaled_preds[show_index][0] > 0.5)[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]

show_comparision(dataset_image_input, dataset_image, prediction, figsize=figsize, show_index=show_index, show_slice=46)
show_comparision(dataset_image_input, dataset_image, prediction, figsize=figsize, show_index=show_index, show_slice=47)


show_index=48
zoom_weight = 74
zoom_weight_horiz = 62
dataset_image_input = (cut_dataset[show_index][0][0])[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]
dataset_image = (cut_dataset[show_index][1][0])[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]
prediction = (rescaled_preds[show_index][0] > 0.5)[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]

show_comparision(dataset_image_input, dataset_image, prediction, figsize=figsize, show_index=show_index, show_slice=47)
show_comparision(dataset_image_input, dataset_image, prediction, figsize=figsize, show_index=show_index, show_slice=48)



show_index=27
zoom_weight = 74
zoom_weight_horiz = 62
dataset_image_input = (cut_dataset[show_index][0][0])[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]
dataset_image = (cut_dataset[show_index][1][0])[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]
prediction = (rescaled_preds[show_index][0] > 0.5)[:, zoom_weight:-zoom_weight, zoom_weight_horiz:-zoom_weight_horiz]

show_comparision(dataset_image_input, dataset_image, prediction, figsize=figsize, show_index=show_index, show_slice=44)

Output()

Output()

Output()

Output()

Output()

Output()

Output()